# 🚀 CenQuery: Llama 3 8B LoRA Fine-Tuning
**Project:** Indian Census Text-to-SQL  
**Hardware:** Runs on T4 GPU (Free Colab Tier)

This notebook fine-tunes the `defog/llama-3-sqlcoder-8b` model on your custom Census dataset using QLoRA (4-bit quantization).
for train_final_(650).jsonl file

## Model Train

### 1. Install Dependencies
We need specific versions of `peft`, `bitsandbytes`, and `transformers` to run 4-bit training.

In [ ]:
!pip install -q -U datasets transformers peft bitsandbytes trl
print("✅ Installation complete. Please RESTART the session now.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.3/512.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 19.4 MB/s eta 0:00:00
✅ Installation complete. Please RESTART the session now.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 2. Upload Training Data
Upload the **`consolidated_train.jsonl`** file that your team created.

In [ ]:
from google.colab import files
import os

# Check if file already exists to avoid re-uploading
if not os.path.exists('consolidated_train.jsonl'):
    print("📂 Please upload 'consolidated_train.jsonl'...")
    uploaded = files.upload()
    # Rename if necessary to ensure it matches the expected filename
    for filename in uploaded.keys():
        if filename != 'consolidated_train.jsonl':
            os.rename(filename, 'consolidated_train.jsonl')
            print(f"   Renamed {filename} to consolidated_train.jsonl")
else:
    print("✅ 'consolidated_train.jsonl' found.")

📂 Please upload 'consolidated_train.jsonl'...


Saving train_S-M-G-N.jsonl to train_S-M-G-N.jsonl
   Renamed train_S-M-G-N.jsonl to consolidated_train.jsonl


### 3. Run QLoRA Training
This script loads the 8B model in 4-bit mode and fine-tunes it on your data.

In [ ]:
from datasets import load_dataset

# 1. Load and Split (Point 1)
dataset = load_dataset("json", data_files="consolidated_train.jsonl", split="train")

# Deterministic split
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"{dataset=}")

# 2. Length Filtering
def filter_long_examples(example, max_chars=8000):
    return len(example["text"]) <= max_chars

dataset["train"] = dataset["train"].filter(filter_long_examples)
dataset["test"] = dataset["test"].filter(filter_long_examples)

# 3. Tokenization & Loss Masking (Point 6)
def tokenize_and_mask(sample):
    # Tokenize the full text
    result = tokenizer(
        sample["text"],
        truncation=True,
        max_length=2048,
        padding=False,
        return_tensors=None
    )

    # Create labels (initially copy of input_ids)
    result["labels"] = result["input_ids"].copy()

    # Find the separator "### SQL" to mask everything before it
    # Note: Adjust the separator string if your tokenizer splits it differently
    sql_header_ids = tokenizer.encode("### SQL", add_special_tokens=False)

    # We look for the start of the SQL section.
    # This loop finds where the SQL part begins in the token list.
    try:
        # Finding the index of the first token of "### SQL"
        # This is a simplified search; robust implementation depends on exact tokenization
        search_token = sql_header_ids[0]
        start_index = result["input_ids"].index(search_token)

        # Mask everything before the SQL part (set to -100)
        # We add len(sql_header_ids) to include the header in the masking if desired,
        # or keep it visible. Usually, we mask the prompt completely.
        mask_len = start_index + len(sql_header_ids)
        result["labels"][:mask_len] = [-100] * mask_len

    except ValueError:
        # If "### SQL" is not found, we mask the whole thing to avoid bad training signal
        result["labels"] = [-100] * len(result["labels"])
        print(f"Value Error in {sample['text']}")

    return result

tokenized_dataset = dataset.map(tokenize_and_mask, remove_columns=dataset["train"].column_names)
print(f"{tokenized_dataset=}")


Generating train split: 0 examples [00:00, ? examples/s]

dataset=Dataset({
    features: ['text'],
    num_rows: 650
})
dataset=DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 585
    })
    test: Dataset({
        features: ['text'],
        num_rows: 65
    })
})


Filter:   0%|          | 0/585 [00:00<?, ? examples/s]

Filter:   0%|          | 0/65 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

Map:   0%|          | 0/19 [00:00<?, ? examples/s]

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "/content/drive/MyDrive/Major_Project/models/llama-3-sqlcoder-8b"
# model_id = "defog/llama-3-sqlcoder-8b"

# 1. Quantization Config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 2. Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fixed alignment for generation

# 3. Load Model with Memory Optimizations
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    use_cache=False, # Disable cache for training (Point 3)
)

# 4. Enable Training Optimizations
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
torch.backends.cuda.matmul.allow_tf32 = True # Speed up on Ampere (if applicable) or generally optimize

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=8,                       # Recommended for small datasets (Point 4)
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj"
    ],
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq

training_args = TrainingArguments(
    output_dir="./lora-sqlcoder-adapter",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=10,
    learning_rate=2e-4,
    warmup_ratio=0.1,
    fp16=True,
    logging_steps=10,

    eval_strategy="epoch",
    save_strategy="epoch",

    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="none",
    optim="paged_adamw_8bit"
)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,0.035200,0.057648
2,0.035600,0.057283
3,0.030300,0.054048
4,0.029600,0.055781
5,0.023700,0.061222
6,0.018900,0.057190
7,0.019400,0.061444
8,0.018300,0.065004
9,0.016700,0.067073
10,0.018700,0.068929


/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/pyt

TrainOutput(global_step=280, training_loss=0.02363238361264978, metrics={'train_runtime': 4332.2959, 'train_samples_per_second': 0.508, 'train_steps_per_second': 0.065, 'total_flos': 4.001398955237376e+16, 'train_loss': 0.02363238361264978, 'epoch': 10.0})

In [ ]:
# Save only the adapters
trainer.save_model("./final_sqlcoder_adapter")
tokenizer.save_pretrained("./final_sqlcoder_adapter")
print("Adapter saved. You can load this on top of the base model for inference.")

Adapter saved. You can load this on top of the base model for inference.


### 4. Save Adapter to Google Drive
Mount your Google Drive to save the trained model permanently. This way, if Colab disconnects, you don't lose your work.

In [ ]:
from google.colab import drive
import shutil
import os

# 2. Define Destination
DESTINATION_FOLDER = "/content/drive/MyDrive/Major_Project/models/CenQuery_Adapter_New_1"

source_dir = "./final_sqlcoder_adapter"

# 3. Copy files
print(f"💾 Copying model files to {DESTINATION_FOLDER}...")
if os.path.exists(DESTINATION_FOLDER):
    shutil.rmtree(DESTINATION_FOLDER)
shutil.copytree(source_dir, DESTINATION_FOLDER)

print("✅ Success! Adapter saved to Google Drive.")

💾 Copying model files to /content/drive/MyDrive/Major_Project/models/CenQuery_Adapter_New_1...
✅ Success! Adapter saved to Google Drive.


## Verify it and Inference

In [1]:
!pip install -q -U transformers peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 11.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
import gc
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# # Define constants if not already defined
BASE_MODEL_NAME = "/content/drive/MyDrive/Major_Project/models/llama-3-sqlcoder-8b"
ADAPTER_PATH = "/content/drive/MyDrive/Major_Project/models/CenQuery_Adapter_New_1"
schema_context = """CREATE TABLE age_groups (id BIGINT PRIMARY KEY, name TEXT);
CREATE TABLE languages (id BIGINT PRIMARY KEY, name TEXT);
CREATE TABLE population_stats (state BIGINT, tru_id BIGINT, age TEXT, persons BIGINT, males BIGINT, females BIGINT);
CREATE TABLE regions (state BIGINT PRIMARY KEY, area_name TEXT);
CREATE TABLE religion_stats (state BIGINT, tru_id BIGINT, religion_id BIGINT, tot_p BIGINT, tot_m BIGINT, tot_f BIGINT);
CREATE TABLE religions (id BIGINT PRIMARY KEY, religion_name TEXT);
CREATE TABLE tru (id BIGINT PRIMARY KEY, name TEXT);"""

In [4]:
def get_started():
    # 1. Clean Memory
    torch.cuda.empty_cache()
    gc.collect()

    print("🚀 Loading Base Model & Adapter (T4-safe)...")

    # 2. 4-bit Config
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )

    # 3. Load Base Model
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_NAME,
        device_map="auto",
        quantization_config=bnb_config,
        dtype=torch.float16,
    )

    # Disable cache to save VRAM and prevent warnings
    base_model.config.use_cache = False

    # 4. Load Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # 5. Load Adapter
    # Note: is_trainable=False is the default, but good to be explicit for inference
    model = PeftModel.from_pretrained(
        base_model,
        ADAPTER_PATH,
        is_trainable=False
    )
    model.eval()

    return tokenizer, model

tokenizer, model = get_started()

🚀 Loading Base Model & Adapter (T4-safe)...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [12]:
def verify_inference_new(question, tokenizer, model, schema_context):
    prompt = f"""### Task
Generate a SQL query to answer the following question:
`{question}`

### Database Schema
{schema_context}

### SQL
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Use terminators to stop Llama-3 from "chatting" after the SQL
    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=False,
            eos_token_id=terminators, # Tells the model to stop at EOT
            pad_token_id=tokenizer.eos_token_id,
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Robust Cleaning:
    # 1. Extract only the text after '### SQL'
    sql_result = full_output.split("### SQL")[-1].strip()

    # 2. Stop at the first mention of 'assistant' or a double newline
    # This removes the "road trip" hallucinations.
    sql_result = sql_result.split("assistant")[0].split("\n\n")[0].strip()

    print(f"🔹 Question: {question}")
    print(f"🔹 Clean SQL: {sql_result}")
    return sql_result

ans = verify_inference_new("How many Muslims live in Kerala?", tokenizer, model, schema_context)
print(f"{ans=}")

🔹 Question: How many Muslims live in Kerala?
🔹 Clean SQL: SELECT rs.tot_p FROM religion_stats rs JOIN regions r ON rs.state = r.state JOIN religions rel ON rs.religion_id = rel.id JOIN tru t ON rs.tru_id = t.id WHERE r.area_name = 'Kerala' AND rel.religion_name = 'Muslim' AND t.name = 'Total';
ans="SELECT rs.tot_p FROM religion_stats rs JOIN regions r ON rs.state = r.state JOIN religions rel ON rs.religion_id = rel.id JOIN tru t ON rs.tru_id = t.id WHERE r.area_name = 'Kerala' AND rel.religion_name = 'Muslim' AND t.name = 'Total';"


## Demo run Tunnel

In [6]:
# 1. Install dependencies
!pip install -q fastapi uvicorn pyngrok nest_asyncio

In [7]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel

# 2. Define the Request Model
class QueryRequest(BaseModel):
    question: str
    schema_context: str | None = None  # Optional if you hardcode it

In [8]:
# 3. Initialize FastAPI
app = FastAPI()

# Enable CORS (Critical for Next.js)
from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allows all origins (safe for demo)
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
def home():
    return {"message": "CenQuery SQLCoder API is Live!"}

@app.post("/generate")
def generate_sql(req: QueryRequest):
    # USE YOUR EXISTING FUNCTION HERE
    # Ensure 'tokenizer' and 'model' are global or loaded before this
    try:
        # Default schema if not provided
        default_schema = """CREATE TABLE religion_stats..."""
        schema = req.schema_context if req.schema_context else default_schema

        sql = verify_inference_new(req.question, tokenizer, model, schema)
        return {"sql": sql}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

In [9]:
# 4. Setup ngrok
# 🔴 REPLACE WITH YOUR TOKEN 🔴
NGROK_AUTH_TOKEN = ""  # ngrok token rotated for security
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# 5. Start the Tunnel
ngrok_tunnel = ngrok.connect(8000)
print('🚀 Public URL:', ngrok_tunnel.public_url)

🚀 Public URL: https://superseraphic-coraline-unspawned.ngrok-free.dev


In [10]:
# Apply the patch (Good practice, even if using await)
nest_asyncio.apply()

# Configure the server
config = uvicorn.Config(app, port=8000)
server = uvicorn.Server(config)

# Start the server in the existing loop
# This cell will stay "Busy" [*] - that is normal!
await server.serve()

INFO:     Started server process [224]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [224]


## Close Tunnel

In [11]:
from pyngrok import ngrok

# 1. Kill all running tunnels
ngrok.kill()

# 2. (Optional) Disconnect specific tunnels if you don't want to kill the process
# ngrok.disconnect("http://localhost:8000")

print("🛑 ngrok tunnels killed. You can now restart the server.")

🛑 ngrok tunnels killed. You can now restart the server.
